<a href="https://colab.research.google.com/github/Al-goritmus/hunty_project/blob/master/preprocess_sources_definitive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##IMPORTAMOS LAS LIBRERÍAS PERTINENTES PARA EL ANÁLISIS Y LA GENERACIÓN DE UN ARCHIVO PARA ENTRENAR EL MODELO DE NLP.

In [1]:
#Librerías generales
import pandas as pd
import numpy as np
import matplotlib.pyplot     as plt
import matplotlib.patches    as mpatches
import seaborn               as sns
import sklearn.metrics       as Metrics
from google.colab import drive
%matplotlib inline

In [2]:
!pip install fuzzywuzzy
!pip install sklearn
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post5-py3-none-any.whl size=2950 sha256=17b4c607a54c230af71bd7bb3d67fccc683dce8f65e36c8170702cfbe3b641bb
  Stored in directory: /root/.cache/pip/wheels/38/1f/8d/4f812c590e074c1e928f5cec67bf5053b71f38e2648739403a
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#Librerías de vectorización y similaridad para procesamiento de texto de manera vectorizada.
import pandas as pd
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
#delcarando part_dir y nombre de archivos
path_dir = '/content/drive/MyDrive/hunty/'
filename = 'vacantes.xlsx'
sheet = 'Sheet1'
filename2 = 'Data Set Users.xlsx'
sheet2 = 'Raw Data'

In [6]:
#data de vacantes
df_vacancies = pd.read_excel(f'{path_dir}{filename}', sheet_name=sheet, header=0)

In [7]:
#data de usuarios
df_users = pd.read_excel(f'{path_dir}{filename2}', sheet_name=sheet2, header=0)

In [8]:
#información general del dataset de vacantes
df_vacancies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   account executive  9999 non-null   float64
 1   area               7461 non-null   object 
 2   work_modality      9560 non-null   object 
 3   country            6530 non-null   object 
 4   city               6529 non-null   object 
 5   remote             10000 non-null  bool   
 6   vacancy_name       10000 non-null  object 
 7   description        10000 non-null  object 
dtypes: bool(1), float64(1), object(6)
memory usage: 556.8+ KB


In [9]:
#información general del dataset de usuarios
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1219 entries, 0 to 1218
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id_user            1219 non-null   int64  
 1   country            1219 non-null   object 
 2   area               1219 non-null   object 
 3   subareas           999 non-null    object 
 4   degrees            3 non-null      object 
 5   wage_aspiration    1218 non-null   float64
 6   currency           1219 non-null   object 
 7   current_wage       347 non-null    float64
 8   change_cities      352 non-null    object 
 9   language           1217 non-null   object 
 10  years_experience   1094 non-null   float64
 11  months_experience  372 non-null    float64
 12  wish_role_name     1008 non-null   object 
 13  work_modality      1219 non-null   object 
 14  hardskills         1186 non-null   object 
dtypes: float64(4), int64(1), object(10)
memory usage: 143.0+ KB


In [10]:
#contenido de la columna descripción
df_vacancies.description

0       Are you looking for a challenging opportunity ...
1       Empresa enfocada en estética médica y dermatol...
2       Sobre el cargo\nEstamos en la búsqueda de WOMe...
3       Gen TI Consulting Somos una empresa mexicana, ...
4       Tienda De Marca Deportiva Número 1 a Nivel Mu...
                              ...                        
9995    Reporta a: COE Manager\nLinio es la tienda en ...
9996    position title: manager, digital marketing\n\n...
9997    importante empresa del sectorrequieredirector ...
9998    location: 100% remotee in mexico – proficient ...
9999    ¡únete a la institución n°1 en microfinanzas d...
Name: description, Length: 10000, dtype: object

In [11]:
#se verifica qué no hay coincidencia directa entre el nombre de la vacante y el rol qué desea ejecutar el usuario
df_vacancies.vacancy_name.isin(df_users.wish_role_name).sum()

0

In [12]:
#Tampoco hay coincidencias entre hardskills y nombre de vacantes
df_vacancies.vacancy_name.isin(df_users.hardskills).sum()

0

In [13]:
#Instalación en máquina virtual de colab
!pip install fuzzywuzzy
!pip install nltk
import nltk
nltk.download('punkt')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import nltk
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    if isinstance(text, str):
        text = text.replace('[^\w\s]', '') 
        text = text.lower()  
        tokens = word_tokenize(text) 
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens] 
        preprocessed_text = ' '.join(lemmatized_tokens)  
        return preprocessed_text
    else:
        return ""

# se aplica la función de preprocesamiento al texto de las columnas wish_role_name y vacancy_name
df_users['preprocessed_wish_role'] = df_users['wish_role_name'].apply(preprocess_text)
df_vacancies['preprocessed_vacancy_name'] = df_vacancies['vacancy_name'].apply(preprocess_text)

# se instancia un vectorizador utilizando TF-IDF
vectorizer = TfidfVectorizer()

# se hace fit_transform utilizando el vectorizador instanciado previamente sobre las columnas de texto preprocesado
X_users = vectorizer.fit_transform(df_users['preprocessed_wish_role'])
X_vacancies = vectorizer.transform(df_vacancies['preprocessed_vacancy_name'])


print(X_users.shape)
print(X_vacancies)
print(df_users['preprocessed_wish_role'])

[nltk_data] Downloading package wordnet to /root/nltk_data...


(1219, 801)
  (0, 311)	0.799648454365415
  (0, 4)	0.6004684416611774
  (1, 617)	0.8545851449778917
  (1, 500)	0.359437124933879
  (1, 239)	0.1761792474152263
  (1, 198)	0.3308323684021669
  (2, 785)	0.5130731278355827
  (2, 699)	0.6222471634673088
  (2, 279)	0.47775648626516226
  (2, 239)	0.34830040608857443
  (3, 793)	0.8115244933612866
  (3, 246)	0.5843184035050643
  (4, 785)	0.30592251048805996
  (4, 754)	0.4669680199457011
  (4, 314)	0.36382484187309855
  (4, 285)	0.2871834502839025
  (4, 239)	0.2076759215282329
  (4, 172)	0.4830059150266994
  (4, 147)	0.23116057877622695
  (4, 52)	0.3790121643564784
  (5, 262)	0.9037557341237794
  (5, 239)	0.4280485638784325
  (6, 696)	0.7960626316751054
  (6, 262)	0.6052142483868879
  (7, 104)	0.7177590318477626
  :	:
  (9991, 311)	0.799648454365415
  (9991, 4)	0.6004684416611774
  (9992, 707)	0.6571694317554144
  (9992, 508)	0.661781078786728
  (9992, 489)	0.36079653785220855
  (9993, 147)	0.520698719006248
  (9993, 52)	0.8537405015724934
  (999

In [15]:
df_vacancies['preprocessed_vacancy_name'].sample(50)

1226          english team leader ( m/f/d ) - mexico city
8407               team leader sale fo unico barranquilla
6244                           sale and comms profesional
5132              group product manager , personalization
4312                                 channel sale manager
5856                       cajero/a auxiliar - villanueva
5497                         director , culture & content
4041                          assistant community manager
3068                         dining service coordinator 1
3079    urgente : asesores de tarjetas de crédito par...
7334                        business intelligence manager
809               customer success manager ( smb+growth )
3841                          sr channel mktng specialist
5310    asesor comercial / call center - proceso inmed...
2542                                      profesional sig
5546              asesores comerciales temporada de madre
8886                                       content writer
1192     brand

In [16]:
df_users['preprocessed_wish_role'].sample(50)

702     gerente comercial , business development manag...
351     analista financiero , analista de tesorería y ...
606       desarrollador backend , desarrollador fullstack
1156    jefe de compras , gerente de compras , supply ...
574     analista de procesos , profesional de procesos...
157     gerente de recursos humanos , gerente desarrol...
126                                                      
274     director comercial , gerente de comercial , ka...
805     analista de marketing , asistente de marketing...
238     director comercial , gerente de cuenta , kam ,...
852     analista logístico de transporte , cadena de s...
845      gerente de proyecto , coordinador de operaciones
848     relaciones publicas , alianzas estratégicas , ...
855         coordinador logisticto , supervisor logistico
1147         supervisor de planta , analista supply chain
782     diseñador gráfico , diseñador ux/ui , desarrol...
1111    líder financiero , project manager finance , l...
31            

In [17]:
!pip install bert-extractive-summarizer


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.6 MB/s eta 0:00:00


In [18]:
df_users['preprocessed_wish_role']

0                                                        
1                                                        
2                                                        
3                                                        
4                                                        
                              ...                        
1214    coordinador de recursos humanos , coordinador ...
1215    especialista recursos humanos , coordinador de...
1216    analista de gestión humana , analista de compe...
1217    asistente administrativo , analista administra...
1218    especialista de compensación , líder de compen...
Name: preprocessed_wish_role, Length: 1219, dtype: object

In [19]:
from summarizer import Summarizer

users_wish_role_column = 'preprocessed_wish_role'
vacancies_description_column = 'preprocessed_vacancy_name'


# Load the pre-trained BERT model
model = Summarizer()

def bert_summarization(text):
    # This ratio determines the amount of information to be retained.
    # Since you're dealing with short texts, you can set it to retain more information.
    ratio = 0.8
    summary = model(text, ratio=ratio)
    return summary

# Apply the summarization to the desired columns
df_users['summary_wish_role'] = df_users[users_wish_role_column].apply(bert_summarization)
df_vacancies['summary_description'] = df_vacancies[vacancies_description_column].apply(bert_summarization)

df_users[['wish_role_name','preprocessed_wish_role','summary_wish_role']].dropna().head(30)


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,wish_role_name,preprocessed_wish_role,summary_wish_role
144,"DIGITAL MEDIA LEAD, DIGITAL MEDIA","digital medium lead , digital medium",
147,AUXILIAR EN OPERACIONES/ GESTION DE COMPRA Y L...,auxiliar en operaciones/ gestion de compra y l...,auxiliar en operaciones/ gestion de compra y l...
148,"CUSTOMER EXPERIENCE ANALYST, ANALISTA DE EXPER...","customer experience analyst , analista de expe...","customer experience analyst , analista de expe..."
149,"DIRECTOR DE ARTE, DIRECTOR GRAFICO","director de arte , director grafico",
150,"COORDINADOR DE CUSTOMER SUCCESS, KEY ACCOUNT M...","coordinador de customer success , key account ...","coordinador de customer success , key account ..."
151,"DIRECTOR CREATIVO, CREATIVE LEAD, HEAD CREATIVO","director creativo , creative lead , head creativo","director creativo , creative lead , head creativo"
152,"GROWTH LEAD, GROWTH MANAGER, CUSTOMER ACQUISIT...","growth lead , growth manager , customer acquis...","growth lead , growth manager , customer acquis..."
153,"CUSTOMER EXPERIENCE, ESPECIALISTA EN CUSTOMER ...","customer experience , especialista en customer...","customer experience , especialista en customer..."
154,"ANALISTA SR CULTURA, ANALISTA SR BIENESTAR, AN...","analista sr cultura , analista sr bienestar , ...","analista sr cultura , analista sr bienestar , ..."
155,"BUSINESS INTELLIGENCE, ANALISTA DE DATOS, BUSI...","business intelligence , analista de datos , bu...","business intelligence , analista de datos , bu..."


In [20]:
df_vacancies['summary_description']

0                                                        
1                                                        
2       ejecutivo de ventas y servicio medellín - 02 d...
3                                                        
4       asesor comercial con experiencia en ventas de ...
                              ...                        
9995                                                     
9996                                                     
9997                                                     
9998                                                     
9999                                                     
Name: summary_description, Length: 10000, dtype: object

In [21]:
df_vacancies['summary_description'].value_counts()

                                                                7689
content strategist ignitetech ( e ) $ 200000/year usd              5
asesor call center - con o sin experiencia , formacion paga        4
digital marketing manager ignitetech ( e ) $ 200000/year usd       3
consultor/a comercial especializado en talento humano              3
                                                                ... 
vendedor/a tat - productos financieros - cartagena                 1
arquitecto de negocio sr ( prevención fraude )                     1
ejecutivo de ventas institucional cali - sector horeca             1
adobe experience cloud manager , digital operation                 1
underwriting product and analytics lead consultant e | wfh         1
Name: summary_description, Length: 2224, dtype: int64

In [22]:
df_users['summary_wish_role'] 

0                                                        
1                                                        
2                                                        
3                                                        
4                                                        
                              ...                        
1214    coordinador de recursos humanos , coordinador ...
1215    especialista recursos humanos , coordinador de...
1216    analista de gestión humana , analista de compe...
1217    asistente administrativo , analista administra...
1218    especialista de compensación , líder de compen...
Name: summary_wish_role, Length: 1219, dtype: object

In [23]:
#!pip install summa

##prueba de sumarización 

In [24]:
#import pandas as pd
#from summa.summarizer import summarize
#
#users_wish_role_column = 'preprocessed_wish_role'
#vacancies_description_column = 'preprocessed_vacancy_name'
#
#
#def text_summarization(text):
#    summary_ratio = 0.95 # este valor establece un ratio entre el tamaño del texto original y la versión resumida
#    summary = summarize(text, ratio=summary_ratio)
#    return summary
#
## se aplica la sumarización a las columnas ['users_wish_role'] del dataframe df_users y ['preprocessed_vacancy_name']
#df_users['summary_wish_role'] = df_users[users_wish_role_column].apply(text_summarization)
#df_vacancies['summary_description'] = df_vacancies[vacancies_description_column].apply(text_summarization)

In [25]:
df_vacancies[['summary_description','description']][df_vacancies['summary_description']!= ''].head(20)

,summary_description,description
2,ejecutivo de ventas y servicio medellín - 02 d...,Sobre el cargo\nEstamos en la búsqueda de WOMe...
4,asesor comercial con experiencia en ventas de ...,Tienda De Marca Deportiva Número 1 a Nivel Mu...
19,"sr brand specialist , consumer electronics",Job Summary\n\nDESCRIPTION\n\nAmazon Mexico Re...
20,profesional en compras y comercio exterior,"Manejo de Importaciones, Exportaciones, Import..."
22,social medium agent ( afternoon/night shift ),"hi, we`re fever\n\nwe`re excited you are check..."
28,ejecutivo de desarrollo de negocios - usa,La marca Open English ha sido líder en el apre...
30,conductor con licencia c2 funza / contratacion...,en eficacia s.a te estamos buscando\n\nimporta...
39,senior marketing automation specialist ( e mex...,bigcommerce’s mission is to help merchants sel...
41,asesor de tecnologia/asesor comercial / bogota...,prestigiosa y reconocida empresa de tecnologí...
42,gran convocatoria auxiliar de enfermería y ad...,¡GRAN CONVOCATORIA!\n\nEste 28 de enero 2023 t...


##output muy pequeño incluso usando ratios altos. Se considera prudente uso de otra librería.

In [26]:
df_users[['wish_role_name','preprocessed_wish_role','summary_wish_role']].dropna().head(30)

,wish_role_name,preprocessed_wish_role,summary_wish_role
144,"DIGITAL MEDIA LEAD, DIGITAL MEDIA","digital medium lead , digital medium",
147,AUXILIAR EN OPERACIONES/ GESTION DE COMPRA Y L...,auxiliar en operaciones/ gestion de compra y l...,auxiliar en operaciones/ gestion de compra y l...
148,"CUSTOMER EXPERIENCE ANALYST, ANALISTA DE EXPER...","customer experience analyst , analista de expe...","customer experience analyst , analista de expe..."
149,"DIRECTOR DE ARTE, DIRECTOR GRAFICO","director de arte , director grafico",
150,"COORDINADOR DE CUSTOMER SUCCESS, KEY ACCOUNT M...","coordinador de customer success , key account ...","coordinador de customer success , key account ..."
151,"DIRECTOR CREATIVO, CREATIVE LEAD, HEAD CREATIVO","director creativo , creative lead , head creativo","director creativo , creative lead , head creativo"
152,"GROWTH LEAD, GROWTH MANAGER, CUSTOMER ACQUISIT...","growth lead , growth manager , customer acquis...","growth lead , growth manager , customer acquis..."
153,"CUSTOMER EXPERIENCE, ESPECIALISTA EN CUSTOMER ...","customer experience , especialista en customer...","customer experience , especialista en customer..."
154,"ANALISTA SR CULTURA, ANALISTA SR BIENESTAR, AN...","analista sr cultura , analista sr bienestar , ...","analista sr cultura , analista sr bienestar , ..."
155,"BUSINESS INTELLIGENCE, ANALISTA DE DATOS, BUSI...","business intelligence , analista de datos , bu...","business intelligence , analista de datos , bu..."


La estrategia de summarize text no parece ser la mejor estrategia con la columna wish_role dada su corta longitud (len()) y la escasa cantidad de output valido incluso subiendo el ratio por encima de 0.5, se utilizará sólo para añadir una versión simplificada de la columna description al dataframe de vacantes, lo siguiente es hacer drop a la columna summary_wish_role 

In [27]:
df_users.drop('summary_wish_role',axis=1,inplace=True)

In [28]:
df_users.columns

Index(['id_user', 'country', 'area', 'subareas', 'degrees', 'wage_aspiration',
       'currency', 'current_wage', 'change_cities', 'language',
       'years_experience', 'months_experience', 'wish_role_name',
       'work_modality', 'hardskills', 'preprocessed_wish_role'],
      dtype='object')

In [29]:
df_vacancies.columns

Index(['account executive', 'area', 'work_modality', 'country', 'city',
       'remote', 'vacancy_name', 'description', 'preprocessed_vacancy_name',
       'summary_description'],
      dtype='object')

##producto de la implementación del fit_transform utilizando el vectorizador instanciado previamente sobre las columnas de texto preprocesado

In [30]:
X_users #matriz para comparación originada en wish_role

<1219x801 sparse matrix of type '<class 'numpy.float64'>'
	with 8265 stored elements in Compressed Sparse Row format>

In [31]:
X_vacancies  #matriz originada en descripción

<10000x801 sparse matrix of type '<class 'numpy.float64'>'
	with 27107 stored elements in Compressed Sparse Row format>

In [32]:
df_vacancies.to_excel('preprocessed_vacancies.xlsx')

In [33]:
df_users.to_excel('preprocessed_users.xlsx')

In [34]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1219 entries, 0 to 1218
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id_user                 1219 non-null   int64  
 1   country                 1219 non-null   object 
 2   area                    1219 non-null   object 
 3   subareas                999 non-null    object 
 4   degrees                 3 non-null      object 
 5   wage_aspiration         1218 non-null   float64
 6   currency                1219 non-null   object 
 7   current_wage            347 non-null    float64
 8   change_cities           352 non-null    object 
 9   language                1217 non-null   object 
 10  years_experience        1094 non-null   float64
 11  months_experience       372 non-null    float64
 12  wish_role_name          1008 non-null   object 
 13  work_modality           1219 non-null   object 
 14  hardskills              1186 non-null   

Habiendo verificado las vectorización del texto para el trabajo deseado y el nombre de la vacante se puede proceder a implementar la similaridad coseno para comparar los vectores originados en las cadenas de texto, al calcular el 'ángulo' entre estos vectores se puede estimar desde un punto de vista de álgebra vectorial el grado de proximidad, similariad o parecido entre estas cadenas de texto, se sintetiza todo el proceso en un sólo código.

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd


#def preprocess_text(text):
#   if isinstance(text, str):
#       text = text.replace('[^\w\s]', '') 
#       text = text.lower()  
#       tokens = word_tokenize(text) 
#       lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens] 
#       preprocessed_text = ' '.join(lemmatized_tokens)  
#       return preprocessed_text
#   else:
#       return ""



def calculate_match(user_pref, vacancy_val): #calculo del match
    if pd.isna(user_pref) or pd.isna(vacancy_val):
        return 0.5  
    if user_pref == vacancy_val:
        return 1  # Full match
    return 0  # No hay match

# Pesos
CITY_MATCH_WEIGHT = 0.1
MODALITY_MATCH_WEIGHT = 0.2

# Preprocessing
#df_users['preprocessed_wish_role'] = df_users['wish_role_name'].apply(preprocess_text)
#df_vacancies['preprocessed_vacancy_name'] = df_vacancies['vacancy_name'].apply(preprocess_text)

# se instancia el vetorizador
vectorizer = TfidfVectorizer()

# se implementa el vectorizador sobre el texto preprocesado
X_users = vectorizer.fit_transform(df_users['preprocessed_wish_role'])
X_vacancies = vectorizer.transform(df_vacancies['preprocessed_vacancy_name'])

similarity_matrix = cosine_similarity(X_users, X_vacancies) # Calculating cosine similarity

top_n_indices = np.apply_along_axis(lambda x: np.argpartition(x, -5)[-5:], 1, similarity_matrix) # se obtiene un top 5 de vacantes similares para cada usuario. 

recommendations_df = pd.DataFrame()

for user_index, vacancy_indices in enumerate(top_n_indices):
    user_id = df_users.loc[user_index, 'id_user']
    user_role = df_users.loc[user_index, 'wish_role_name']

    for vacancy_index in vacancy_indices:
        vacancy_name = df_vacancies.loc[vacancy_index, 'vacancy_name']
        vacancy_description = df_vacancies.loc[vacancy_index, 'description']
        vacancy_summary = df_vacancies.loc[vacancy_index, 'summary_description']
        vacancy_work_modality = df_vacancies.loc[vacancy_index, 'work_modality']

        # ciudad y modalidad del match
        city_match = calculate_match(df_users.loc[user_index, 'change_cities'], df_vacancies.loc[vacancy_index, 'city'])
        modality_match = calculate_match(df_users.loc[user_index, 'work_modality'], df_vacancies.loc[vacancy_index, 'work_modality'])

        # sumatoria con pesos para las variables y  establecer el match al multiplicar score de similaridad por ponderación
        similarity_score = similarity_matrix[user_index, vacancy_index]
        total_match = similarity_score * (1 - CITY_MATCH_WEIGHT - MODALITY_MATCH_WEIGHT) + city_match * CITY_MATCH_WEIGHT + modality_match * MODALITY_MATCH_WEIGHT

        # Append al dataframe
        recommendations_df = recommendations_df.append({
            'id_user': user_id,
            'wish_role_name': user_role,
            'vacancy_name': vacancy_name,
            'description': vacancy_description,
            'summary_description': vacancy_summary,
            'work_modality': vacancy_work_modality,
            'total_match': total_match
        }, ignore_index=True)

print(recommendations_df)


Se truncaron las últimas líneas 5000 del resultado de transmisión.
<ipython-input-36-0b73bdaa6cfd>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append({
<ipython-input-36-0b73bdaa6cfd>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append({
<ipython-input-36-0b73bdaa6cfd>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append({
<ipython-input-36-0b73bdaa6cfd>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append({
<ipython-input-36-0b73bdaa6cfd>:59: FutureWarning: Th

      id_user                                     wish_role_name  \
0           1                                                NaN   
1           1                                                NaN   
2           1                                                NaN   
3           1                                                NaN   
4           1                                                NaN   
...       ...                                                ...   
6090     1219  ESPECIALISTA DE COMPENSACIÓN,LÍDER DE COMPENSA...   
6091     1219  ESPECIALISTA DE COMPENSACIÓN,LÍDER DE COMPENSA...   
6092     1219  ESPECIALISTA DE COMPENSACIÓN,LÍDER DE COMPENSA...   
6093     1219  ESPECIALISTA DE COMPENSACIÓN,LÍDER DE COMPENSA...   
6094     1219  ESPECIALISTA DE COMPENSACIÓN,LÍDER DE COMPENSA...   

                                           vacancy_name  \
0                   customer success manager   (mexico)   
1                                           f&b manager   
2     

<ipython-input-36-0b73bdaa6cfd>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append({
<ipython-input-36-0b73bdaa6cfd>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append({
<ipython-input-36-0b73bdaa6cfd>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append({
<ipython-input-36-0b73bdaa6cfd>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append({
<ipython-input-36-0b73bdaa6cfd>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas

#Este dataframe toma en cuenta otras variables cómo la modalidad de trabajo, la descripción y la ciudad para hacer recomendaciones

In [38]:
recommendations_df

,id_user,wish_role_name,vacancy_name,description,summary_description,work_modality,total_match
0,1,NaN,customer success manager (mexico),ForceManager is the sales CRM for teams that a...,,Tiempo completo,0.250000
1,1,NaN,f&b manager,description\n\n\n\n\nwho we are\n\n\n\n\nselin...,,Tiempo completo,0.250000
2,1,NaN,elementary/middle school english teacher,teacher with a bachelor’s degree in education ...,,Tiempo completo,0.250000
3,1,NaN,asesor comercial punto de venta retail sector ...,en eficacia buscamos nuevos talentos¡\n\nreque...,asesor comercial punto de venta retail sector ...,Tiempo completo,0.250000
4,1,NaN,especialista de gestión efectiva,¡únete a la institución n°1 en microfinanzas d...,,Tiempo completo,0.250000
...,...,...,...,...,...,...,...
6090,1219,"ESPECIALISTA DE COMPENSACIÓN,LÍDER DE COMPENSA...",hr systems compensation & benefits system analyst,HP is a global brand with over 75 years in bus...,hr system compensation & benefit system analyst,Tiempo completo,0.261456
6091,1219,"ESPECIALISTA DE COMPENSACIÓN,LÍDER DE COMPENSA...",compensation analyst (e),job description\n\njob description\n\noverview...,,Tiempo completo,0.381501
6092,1219,"ESPECIALISTA DE COMPENSACIÓN,LÍDER DE COMPENSA...",compensation analyst e,about hireright\n\n\n\n\nhireright is the prem...,,Tiempo completo,0.381501
6093,1219,"ESPECIALISTA DE COMPENSACIÓN,LÍDER DE COMPENSA...",Sales Compensation Specialist,La Haus tiene como visión volver accesible la ...,,NaN,0.482600


In [ ]:
## se aplica preprocesamiento a las columnas originales
#df_users['preprocessed_wish_role'] = df_users['wish_role_name'].apply(preprocess_text)
#df_vacancies['preprocessed_vacancy_name'] = df_vacancies['vacancy_name'].apply(preprocess_text)
#
## se instancia el vetorizador.
#vectorizer = TfidfVectorizer()
#
## se implementa el vectorizador sobre las columnas de texto preprocesado.
#X_users = vectorizer.fit_transform(df_users['preprocessed_wish_role'])
#X_vacancies = vectorizer.transform(df_vacancies['preprocessed_vacancy_name'])
#
#
#similarity_matrix = cosine_similarity(X_users, X_vacancies) # se calcula la similaridad coseno entre estos vectores. 
#
#top_n_indices = np.apply_along_axis(lambda x: np.argpartition(x, -5)[-5:], 1, similarity_matrix) # se obtiene un top 5 de vacantes similares para cada usuario. 
#
#recommendations_df = pd.DataFrame() #se crea un dataframe vacío dónde vamos a sintetizar la data correspondiente al entrenamiento del modelo.
#
## se itera sobre cada usuario y sus top de vacancia relativos originados en la matriz de similaridad usando np.argpartition con el fin de no aumentar la complejidad temporal del algoritmo.  
#for user_index, vacancy_indices in enumerate(top_n_indices):
#    user_id = df_users.loc[user_index, 'id_user']
#    user_role = df_users.loc[user_index, 'wish_role_name']
#
#    for vacancy_index in vacancy_indices:
#        vacancy_name = df_vacancies.loc[vacancy_index, 'vacancy_name']
#        vacancy_description = df_vacancies.loc[vacancy_index, 'description']
#        vacancy_summary = df_vacancies.loc[vacancy_index, 'summary_description']
#        vacancy_work_modality = df_vacancies.loc[vacancy_index, 'work_modality']
#
#        #se constriiye el dataframe haciendo un append con las variables previamente declaradas con información relevante de las  vacantes 
#        recommendations_df = recommendations_df.append({
#            'id_user': user_id,
#            'wish_role_name': user_role,
#            'vacancy_name': vacancy_name,
#            'description': vacancy_description,
#            'summary_description': vacancy_summary,
#            'work_modality': vacancy_work_modality
#        }, ignore_index=True)
#
## se imprime el top 5 de recomendaciones para cada usuario ahora condensado en un sólo dataframe
#print(recommendations_df)
#

In [ ]:
recommendations_df.info() #primera visión del dataset de entrenamiento

In [ ]:
recommendations_df.head(50) #primeras 50 filas del dataset. Se evidencia la necesidad de manejar los valores nulos en la columna wish_role_name, qué no superan el 20% de los valores

In [ ]:
recommendations_df.dropna(subset='wish_role_name') #dado qué esta columna funcionará cómo el prompting para el modelo e igual en el entrenamiento, se elimina las filas en las cual este valor es nulo
#Pandas improme una coipia del dataframe cuando no usamos df = df... o inplace=True.

In [ ]:
recommendations_df.dropna(subset='wish_role_name',inplace=True)

In [ ]:
recommendations_df.to_excel('dataset_entrenamiento1.xlsx')

#Por ultimo, sabemos qué los modelos GPT cómo el qué vamos a implementar en este proyecto son modelos qué reciben de manera preferencial versiones tokenizadas de texto en un formato  prompt + completion y para este fin el formato json es el ideal, se procede a correr código en python con el fin de generar un json formateado con el contenido 

In [ ]:
import pandas as pd
import json

# leer el dataset de recomendaciones en caso de utilizar este cómo fuente.
# df = pd.read_excel('data_entrenamiento1.xlsx')  # en caso de utilizar un dataset qué fue previamente almacenado en un exce, es decir si recommendations_df es un excel.

# Create a list of dictionaries with the desired format
data = []
for _, row in recommendations_df.iterrows():
    # considerando el uso de 'wish_role_name' cómo prompt
    prompt = row['wish_role_name']
    
    # este ciclo for concatena las variables contenidas en las columnas ['vacancy_name', 'description', and 'summary_description'] as completion
    # se añaden tokens especiales para indicar al modelo el inicio y final de cada campo 
    completion = "<vacancy_name_start> " + row['vacancy_name'] + " <vacancy_name_end> " + \
                 "<description_start> " + row['description'] + " <description_end> " + \
                 "<summary_description_start> " + row['summary_description'] + " <summary_description_end>"
    data.append({"prompt": prompt, "completion": completion})

# El ciclo for anterior genera una lista de diccionarios ahora los incluimos en un JSON con encoding = 'Unicode'
with open('primer_entrenamiento.jsonl', 'w', encoding='utf-8') as f:
    for item in data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')
